In [1]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz --user
nlp = en_core_web_sm.load()
import pandas as pd
import json
!pip install bs4 --user
from bs4 import BeautifulSoup
import requests
import re
import os
import mmap

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Load data

In [5]:
with open('data.json', 'r') as f:
    dict_json_weighted = json.load(f)
object_1 = list(dict_json_weighted.keys())[4]
for i in range(len(dict_json_weighted[object_1]['log'])):
    print(str(i)+"-"+dict_json_weighted[object_1]['log'][i]['text'])

0-Hello, I am looking for a cheap place to dine in the centre.
1-We have lots of those!  Is there a particular type of food you'd like?
2-not really but i need it for 3 people
3-I recommend La Raza - fantastic (and cheap!) Spanish cuisine. I'd be happy to book a table for you, if you like.
4-Yes for 3 at 14:45 wednesday, can I get the reference number too please
5-Booking was successful. The table will be reserved for 15 minutes. Reference number is : IBTN1DDC.
6-Thanks. I also need a cheap place to stay in the center.
7-That gives you two options. Both are guesthouses. Both offer free parking and free wifi. The Alexander B&B is 4 star rated. The El Shaddai has a 0 rating.
8-What is the price of the Alexander?
9-It is a cheap place.
10-ok, i need to book it for the same day.
11-Okay for how many nights and how many people?
12-3 nights, 3 people, please.
13-You are all booked for the alexander bed and breakfast! Your reference number is 66XCD1XW. Is there anything else you need?
14-I wo

## Load ontology

In [6]:
with open('ontology.json', 'r') as o:
    ontology = json.load(o)
action=['inform', 'request', 'select', 'recommend', 'not found', 'request booking', 'offer booking', 
        'inform booked', 'decline booking', 'welcome','greet', 'bye', 'reqmore' ]
topic= ['hotel', 'restaurant', 'taxi', 'train', 'attraction', 'police', 'hospital']

## display topics and actions

In [12]:
top=[]
act=[]
ont=list(ontology.keys())
for i in range(len(dict_json_weighted[object_1]['log'])):
    #print(dict_json_weighted[object_1]['log'][i]['text'])
    sentence =nlp (dict_json_weighted[object_1]['log'][i]['text'])
    for x in [y for y in sentence if y.pos_ != 'PUNCT']:
        if str(x).lower() in topic:
            if str(x).lower() not in top:
                top.append(str(x).lower()) 
        if x.lemma_ in action:
            if x.lemma_ not in act:
                act.append(x.lemma_)
        #for z in ont:
            #if str(x) in str(ontology[z]):
                #print(z+" :"+str(x))           
print("topics: "+str(top))
print("actions: "+str(act))


topics: ['taxi', 'hotel', 'restaurant']
actions: ['recommend', 'welcome', 'bye']


## Display the updated slots 

In [11]:
for i in dict_json_weighted[object_1]['goal']:
    if i not in['message', 'topic']:
        if dict_json_weighted[object_1]['goal'][i] !={}:
            print("****** "+i+" ******")
            for j in (dict_json_weighted[object_1]['goal'][i]):
                if dict_json_weighted[object_1]['goal'][i][j] not in ['fail_info', {}]:
                    print("°"+j)
                    print(dict_json_weighted[object_1]['goal'][i][j])
    

****** taxi ******
°info
{'arriveBy': '14:45'}
°reqt
['car type', 'phone']
****** restaurant ******
°info
{'pricerange': 'cheap', 'area': 'centre'}
°fail_info
{'pricerange': 'expensive', 'area': 'north'}
°book
{'day': 'wednesday', 'people': '3', 'invalid': False, 'time': '14:45'}
****** hotel ******
°info
{'internet': 'yes', 'pricerange': 'cheap', 'area': 'centre'}
°book
{'stay': '3', 'day': 'wednesday', 'people': '3', 'invalid': False}


# Custom NER

### create the ner pipeline component and add it to model

In [42]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe('ner')

In [93]:
## adding entities
for ent in ontology:
    ner.add_label(ent)

other_pipe = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

# Only training NER
with nlp.disable_pipes(*other_pipe) :
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()

### train the model

In [95]:
for y in range(10) :
    print("Starting iteration" + str(y))
    losses = {}
    for i in dict_json_weighted[object_1]['log']:
        nlp.update(
        [dict_json_weighted[object_1]['log'][i]['metadata']],
        [dict_json_weighted[object_1]['log'][i]['text']],
        drop = 0.2,
        sgd = optimizer,
        losses = losses
        )
      #print(losses)
new_model = nlp

Starting iteration0


TypeError: list indices must be integers or slices, not dict